[paxos simple](https://lamport.azurewebsites.net/pubs/paxos-simple.pdf)
-------

### 1.1 共识算法 - consensus algorithm

#### 一. 问题提出
1. 假设有一个进程集合, 每个进程都能提出一个值(propose a value). 因此, 这个集合会提出多个不同的值.   
 共识算法, 就是要从这些建议的值中, 选择一个作为最终结果. 且集合中的每个进程都能学习到这个最终值
2. 共识算法的三种agent角色  
  1. proposers
  2. acceptors
  3. learners  
工程实现上, 3个角色并非分成独立的进程, 一个进程可以同时是这3种角色

#### 二. 共识算法的基本假设  
1. 假设**三种agent角色的处理速度任意快慢**, 并可能宕机重启. 所有的agent都可能在最终值产生后宕机和重启, 这种情况只能让agent能够记录自己宕机和重启前的值  
2. 假设agent之间可以传递任意长的消息, 且**消息可能由于网络原因而导致重复, 丢失**, 但**消息不会被篡改**而错乱



### 1.2 如何选取值
#### 一. 最简单的情形 - 只有一个acceptor agent  
1. 这种情况只需让acceptor agent选择第一次收到的来自于某个proposer的提案即可.  
 但1个acceptor agent会带来单点故障问题, 失去分布式的安全性
 
#### 二. 多acceptor agent的情形  
1. 每个proposer agent把建议值发给一组acceptor, 而每个acceptor只能接受其中一个proposer agent的建议值  
2. 当有足够多的accpt agent接受了同一个建议值时, 这个值就作为最终选择的值  
3. 在不考虑消息丢失,宕机重启的情况下, 为了确保最终能选出一个值, 做出如下要求:  
    ```shell
    p1: 每个accptor必须接受第一个收到的建议值
    ```    
  但这会带来新问题, 假如有2个不同的建议值, 分别被一半的accptor接受, 则无法选取最终值. 因此, 必须让acceptor能够接受多个提案(propposal). 为避免混淆, 为每个提案(propposal)添加一个编号用来区分. 此后, 对于不同建议值各被一半accptor接受的情形, 我们可以选择2个提案(propposal)作为最终值, 但要求这两个提案(propposal)包含的值相同. 因此, 又做出第二个要求:  
    ```shell
    p2: 如果一个值为v的提案(propposal)已被接受, 则每个序号更大的提案(propposal)也必须值为v
    ```
 从acceptor的角度看, 得出等价的要求`p2a`  
    ```shell
    p2a: 如果一个值为v的提案(propposal)已被接受, 则每个序号更大的, 被任何其他acceptor接受的提案(propposal)也必须值为v
    ```
 从proposaler的角度看, 得出等价要求`p2b`  
    ```shell
    p2b: 如果一个值为v的提案(propposal)已被接受, 则每个序号更大的, 被任何其他proposaler提出的提案(propposal)也必须值为v  
    ```
 以上, 我们可以总结, 若编号为n,值为v的提案(propposal)被选择成最终结果, 则acceptor有如下性质  
    ```shell
    p2c: 一半以上的acceptor构成的集合S:  
     要么S中没有1个acceptor接受过编号小于n的提案(propposal)  
     要么S中每个Acceptor接受过的最大编号的提案的值为V。
    ```
    
#### 三.  Proposer请求步骤 - 如何保证p2b的实现
1. 先进行prepare请求:    
 首先,proposer选择一个新的编号N, 随机选择一半以上的acceptor并向他们发出请求, 要求这些acceptor满足:  
   1. 不再接受编号小于N的提案  
   2. 如果收到该prepare请求的acceptor已接受过提案, 则向请求的proposer返回编号小于N的最大编号的提案中包含的值  
2. 在进行accept请求:   
 如果proposer的prepare请求, 由半数以上的accpter做出回应, 则proposer可以生成编号为n,值为v的提案(propposal).这里的v满足:  
   1. 若所有响应中都没有提案, 说明这些accepter从没接受过提案(propposal), 可以设置v  
   2. 否则, 选择返回的请求中, 编号最大的提案的值作为v  
 构造好提案(propposal)后, 再次随机选择半数以上的accepter请求该提案. 这次的请求称为accept请求
 
#### 四. Acceptor接受提案的步骤 - 保证p2a的实现
1. Acceptor可以忽略所有请求, 包括Proposer的prepare请求和accept请求. 这并不影响Proposer的运行. 因此我们只关注什么时候Acceptor会响应请求, 即:  
 ```bash
  只要Acceptor未响应过任何编号大于N的Prepare请求，那么他就可以接受这个编号为N的提案。
  ```
2. 根据p2a的要求, 如果Acceptor已经响应过编号大于n的prepare请求, 则Acceptor就不能接收编号为n的提案(propposal).  
 此时Acceptor可直接忽略请求或返回error响应. 因此, Acceptor只需记住:  
   1. 已接受的编号最大的提案(值)  
   2. 已响应的编号最大的prepare请求的编号

### 1.3 完整的Paxos算法步骤

#### 阶段一
1. Proposer产生编号为n的prepare请求, 发往半数以上的Acceptor  
2. 每个接收到prepare请求的Acceptor, 返回自己已接受的提案(proposal)的最大编号(若从未接收过提案则返回空), 并承诺不再接受编号小于n的accept提案请求

#### 阶段二
1. Proposer从返回的半数以上Accptor返回的prepare响应提取出编号最大的提案(proposal)的值v, 构建accept请求(编号n,值v)再发往半数以上的acceptor. 其中, 若所有返回的prepare响应中均不包含带编号的提案, 则v可自由指定.  
2. 每个Acceptor从收到的accpt请求中提取编号n, 若n大于等于自己记录的"已响应的最大prepare请求编号", 则接受这个请求, 否则忽略此次请求  

#### 阶段三
阶段三为: learner学习选出的最终value. 有3种方式:  
1. Acceptor每接受一个提案, 就向所有learner发送该提案:  
  优点: learner可快速获取被选定的value    
  缺点: 通信次数为(accpter个数\*learner个数)
2. 只有一个主learner, 每个Acceptor都把接受的值发送给主learner  
  优点: 减少通信量  
  缺点: 单点故障  
3. Acceptor每接受一个提案, 就向一个learner集合发送该提案, 然后该集合再通知其它learner  
  优点: 集合中learner个数越多, 越可靠  
  缺点: 通信量增加 

### 1.4 如何保证Paxos的活性  
假设有2个proposer依次提出编号递增的提案, 则可能发生无限循环的可能, 且没有value被选定. (无法保证活性)
<img src="./img/huoxing" width="85%" height="85%">
因此, 必须选择一个主Proposer, 只有主Proposer能提出提案. 至此，我们得到一个既能保证安全性，又能保证活性的分布式一致性算法——Paxos算法。